In [1]:
import pandas as pd

In [4]:
messages = pd.read_csv('SMSSpamCollection', sep='\t',
                           names=["label", "message"])

In [5]:
#Data cleaning and preprocessing
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [7]:
ps = PorterStemmer()
corpus = []

In [8]:
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [9]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()

y=pd.get_dummies(messages['label'])
y=y.iloc[:,1].values

In [10]:
# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [11]:
# Training model using Naive bayes classifier

from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

y_pred=spam_detect_model.predict(X_test)

In [13]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_pred,y_test)
cm

array([[946,   7],
       [  9, 153]])

In [19]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred,y_test)
score

0.9856502242152466

In [20]:
import pickle
saved_model = pickle.dumps(spam_detect_model) 

In [21]:
saved_model

b'\x80\x03csklearn.naive_bayes\nMultinomialNB\nq\x00)\x81q\x01}q\x02(X\x05\x00\x00\x00alphaq\x03G?\xf0\x00\x00\x00\x00\x00\x00X\t\x00\x00\x00fit_priorq\x04\x88X\x0b\x00\x00\x00class_priorq\x05NX\x0b\x00\x00\x00n_features_q\x06M\xc4\tX\x08\x00\x00\x00classes_q\x07cnumpy.core.multiarray\n_reconstruct\nq\x08cnumpy\nndarray\nq\tK\x00\x85q\nC\x01bq\x0b\x87q\x0cRq\r(K\x01K\x02\x85q\x0ecnumpy\ndtype\nq\x0fX\x02\x00\x00\x00u1q\x10K\x00K\x01\x87q\x11Rq\x12(K\x03X\x01\x00\x00\x00|q\x13NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x14b\x89C\x02\x00\x01q\x15tq\x16bX\x0c\x00\x00\x00class_count_q\x17h\x08h\tK\x00\x85q\x18h\x0b\x87q\x19Rq\x1a(K\x01K\x02\x85q\x1bh\x0fX\x02\x00\x00\x00f8q\x1cK\x00K\x01\x87q\x1dRq\x1e(K\x03X\x01\x00\x00\x00<q\x1fNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq b\x89C\x10\x00\x00\x00\x00\x00<\xae@\x00\x00\x00\x00\x00X\x82@q!tq"bX\x0e\x00\x00\x00feature_count_q#h\x08h\tK\x00\x85q$h\x0b\x87q%Rq&(K\x01K\x02M\xc4\t\x86q\'h\x1e\x89B@\x9c\x00\x00\x00\x00\x00\x00\x00\x00\x08@\x00\x0

In [22]:
import pickle

# Save trained model to file
pickle.dump(spam_detect_model, open("spam_classifier.pkl", "wb"))